In [1]:
import os
from datetime import datetime
import requests
import pandas as pd
from dotenv import load_dotenv
import pyarrow.feather as feather


In [2]:
load_dotenv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/env/usdoe.env')
api_key = os.getenv('COLLEGE_SCORECARD_API_KEY')

example field selection request:
- https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&fields=id,school.name,latest.cost.tuition.in_state,latest.completion.rate

search by name:
- https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&school.name=Stanford University

search by state and city:
- https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&school.state=NY&school.city=New York

specific fields:
- https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&fields=id,school.name,latest.admissions.admission_rate.overall,latest.student.size

paginated requests:
- https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&school.state=TX&page=2&per_page=10


school.degrees_awarded.predominant=2,3,4 will match records with a school.degrees_awarded.predominant value of 2, 3 or 4

Append the characters __not to the parameter name to specify a negative match.
For example: school.region_id__not=5 matches on records where the school.region_id does not equal 5

To match on field values in a particular numeric range, use a range match. Append the characters __range to the parameter name to specify a range match, and provide two numbers separated by two periods (..).
For example: 2013.student.size__range=100..500 matches on schools which had between 100 and 500 students in 2013.
Open-ended ranges can be performed by omitting one side of the range. For example: 2013.student.size__range=1000.. matches on schools which had over 1000 students.

Geographic Filtering with zip and distance
When the dataset includes a location at the root level (location.lat and location.lon) then the documents will be indexed geographically. You can use the zip and distance options to narrow query results down to those within a geographic area. For example, zip=12345&distance=10mi will return only those results within 10 miles of the center of the given zip code.
Additionally, you can request location.lat and location.lon in a search that includes a fields filter and it will return the record(s) with respective lat and/or lon coordinates.

Additional Notes on Geographic Filtering
By default, any number passed in the distance parameter is treated as a number of miles, but you can specify miles or kilometers by appending mi or km respectively.
Distances are calculated from the center of the given zip code, not the boundary.
Only U.S. zip codes are supported

Nesting Field of Study Results
The field of study data is included as an array of objects nested under a specified key. These objects may be queried just like any other data. However, there is an additional parameter to add to your API call to manage what is returned. By default, if specifying a search parameter, only objects of the array that match that parameter will be returned. You can pass &all_programs_nested=true to return all the items in the array instead of just those that match.

In [4]:
# Function to get user input for criteria
def get_user_input():
    # Prompt for specific fields to include in the dataset
    fields = input("Enter the fields you want (comma-separated, e.g., id,school.name,latest.student.size): ")
    
    # Prompt for filtering criteria
    state = input("Enter state (e.g., CA) or leave blank for all: ")
    size = input("Enter student size range (e.g., '1000..5000' for 1000 to 5000) or leave blank for all: ")
    tuition = input("Enter max out-of-state tuition (e.g., 30000) or leave blank for all: ")
    fields_of_study = input("Enter field of study (CIP code) or leave blank for all: ")
    
    # Construct the parameters dictionary based on user input
    params = {
        'api_key': api_key,
        'fields': fields if fields else 'id,school.name,school.city,school.state,latest.student.size,latest.cost.tuition.out_of_state,latest.admissions.admission_rate.overall',
        'per_page': 100,
        'page': 0
    }
    if state:
        params['school.state'] = state
    if size:
        params['latest.student.size__range'] = size
    if tuition:
        params['latest.cost.tuition.out_of_state__lte'] = tuition
    if fields_of_study:
        params['latest.programs.cip_4_digit.code'] = fields_of_study
    
    return params

def fetch_data(params):
    base_url = "https://api.data.gov/ed/collegescorecard/v1/schools"
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    return data['results']

# Function to extract and flatten field of study data
def extract_fields_of_study(data):
    results = []
    for school in data:
        if 'latest.programs.cip_4_digit' in school:
            for program in school['latest.programs.cip_4_digit']:
                record = {**school, **program}
                results.append(record)
    return results

# Function to save data to Feather file
def save_data_to_feather(data, filename_prefix="college_scorecard_data"):
    df = pd.DataFrame(data)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'/Users/paigeblackstone/Library/Mobile Documents/com~apple~CloudDocs/us_doe_data/{filename_prefix}_{timestamp}.feather'
    feather.write_feather(df, filename)
    print(f"Data saved to {filename}")

# Main function
def main():
    params = get_user_input()
    data = fetch_data(params)
    
    # Check if field of study data should be included
    if 'latest.programs.cip_4_digit.code' in params:
        extracted_data = extract_fields_of_study(data)
        save_data_to_feather(extracted_data, filename_prefix="field_of_study_data")
    else:
        save_data_to_feather(data)

# Run the script
if __name__ == "__main__":
    main()

Data saved to /Users/paigeblackstone/Library/Mobile Documents/com~apple~CloudDocs/us_doe_data/college_scorecard_data_20240801_145755.feather


In [7]:
filename = '/Users/paigeblackstone/Library/Mobile Documents/com~apple~CloudDocs/us_doe_data/college_scorecard_data_20240801_145755.feather'

# Read the Feather file into a DataFrame
df = pd.read_feather(filename)

# Preview the DataFrame
print(df.head())  # Display the first few rows

   latest.student.size  latest.cost.tuition.out_of_state  \
0                 5196                             18634   
1                12776                             21216   
2                  228                                 0   
3                 6985                             24770   
4                 3296                             19396   

   latest.admissions.admission_rate.overall  \
0                                    0.6840   
1                                    0.8668   
2                                       NaN   
3                                    0.7810   
4                                    0.9660   

                           school.name school.city school.state      id  
0             Alabama A & M University      Normal           AL  100654  
1  University of Alabama at Birmingham  Birmingham           AL  100663  
2                   Amridge University  Montgomery           AL  100690  
3  University of Alabama in Huntsville  Huntsville          